# Query 1

### Display list of stays in Portland, OR with details: name, neighbourhood, room type, how many guests it accommodates, property type and amenities, per night’s cost and is available for the next two days in descending order of rating. 

In [1]:
#importing functions
import numpy as np
import pandas as pd

# Read the data and store in a Pandas DataFrame
df_calendar = pd.read_csv("calendar.csv")
df_listings = pd.read_csv("listings.csv")

In [2]:
#delete all columns except id, name, neighborhood_cleansed, property_type, room_type, accomodates, amenities and price from listing.csv
df_listings = df_listings[['id', 'name', 'neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates', 'amenities', 'price', 'review_scores_rating']]

#rename the review_scores_rating
df_listings.rename(columns={'review_scores_rating': 'rating'}, inplace=True)

#convert id to int
df_listings['id'] = df_listings['id'].astype(int)

In [3]:
#delete all columns except listing_id, date and available from calendar.csv
df_calendar = df_calendar[['listing_id', 'date', 'available']]

#convert listing_id to int
df_calendar['listing_id'] = df_calendar['listing_id'].astype(int)

In [4]:
#join df_calendar and df_listings on listing_id
df_stays = pd.merge(df_calendar, df_listings, left_on='listing_id', right_on='id')

#remove the column id
df_stays = df_stays.drop(columns=['id'])
df_stays.head()


,listing_id,date,available,name,neighbourhood_cleansed,property_type,room_type,accommodates,amenities,price,rating
0,12899,3/23/23,f,"Alberta Arts 2 bedroom suite, charming 1906 house",Concordia,Entire rental unit,Entire home/apt,3,"[""Carbon monoxide alarm"", ""Refrigerator"", ""Smo...",$80.00,4.93
1,12899,3/24/23,f,"Alberta Arts 2 bedroom suite, charming 1906 house",Concordia,Entire rental unit,Entire home/apt,3,"[""Carbon monoxide alarm"", ""Refrigerator"", ""Smo...",$80.00,4.93
2,12899,3/25/23,f,"Alberta Arts 2 bedroom suite, charming 1906 house",Concordia,Entire rental unit,Entire home/apt,3,"[""Carbon monoxide alarm"", ""Refrigerator"", ""Smo...",$80.00,4.93
3,12899,3/26/23,f,"Alberta Arts 2 bedroom suite, charming 1906 house",Concordia,Entire rental unit,Entire home/apt,3,"[""Carbon monoxide alarm"", ""Refrigerator"", ""Smo...",$80.00,4.93
4,12899,3/27/23,f,"Alberta Arts 2 bedroom suite, charming 1906 house",Concordia,Entire rental unit,Entire home/apt,3,"[""Carbon monoxide alarm"", ""Refrigerator"", ""Smo...",$80.00,4.93


In [5]:
from datetime import datetime, timedelta

# Get the current date
current_date = datetime.now().date()

one_day_from_now = current_date + timedelta(days=1)
two_days_from_now = current_date + timedelta(days=2)

one_day_from_now = one_day_from_now.strftime("%m-%d-%Y")
two_days_from_now = two_days_from_now.strftime("%m-%d-%Y")

print(one_day_from_now)
print(two_days_from_now)

06-05-2023
06-06-2023


In [6]:
#convert date column to strftime
df_stays['date'] = pd.to_datetime(df_stays['date'])
df_stays['date'] = df_stays['date'].dt.strftime('%m-%d-%Y')

#convert df_stays to csv
df_stays.to_csv('stays.csv', index=False)

In [7]:
df = pd.read_csv("stays.csv")

In [8]:
#filter the data for a listing having one_day_from_now and two_days_from_now where date available is 't'
df = df[(df['date'] == one_day_from_now) | (df['date'] == two_days_from_now)]
df = df[(df['available'] == 't')]

In [9]:
test = df.groupby('listing_id').count()['date'].reset_index()
final_ids = test[test.date == 2]['listing_id']
final_ids

0                  1386231
1                  1386922
2                  1408895
4                  1525272
5                  1529860
               ...        
1859    721010000000000000
1860    721417000000000000
1861    721645000000000000
1862    721648000000000000
1863    721652000000000000
Name: listing_id, Length: 1746, dtype: int64

In [10]:
#sort in descending order of review_scores_rating
df = df.sort_values(by=['rating'], ascending=False)

#filter df listing_id with final_ids
df = df[df['listing_id'].isin(final_ids)]

df = df.drop_duplicates(subset=['listing_id'])
df = df.drop(columns=['date', 'available']).reset_index(drop = True)

df

,listing_id,name,neighbourhood_cleansed,property_type,room_type,accommodates,amenities,price,rating
0,54342436,Cheerful 2 Bedroom Home. Walk Score: 91 & Parking,Sunnyside,Entire home,Entire home/apt,5,"[""Carbon monoxide alarm"", ""HDTV with Chromecas...",$137.00,5.0
1,668894000000000000,Urban Luxury Near Williams & Alberta w/ Parking,Humboldt,Entire townhouse,Entire home/apt,4,"[""Carbon monoxide alarm"", ""Refrigerator"", ""Smo...",$208.00,5.0
2,13744593,Mt. Tabor Elegant Cottage,Mt. Tabor,Entire home,Entire home/apt,2,"[""Carbon monoxide alarm"", ""Children\u2019s boo...",$89.00,5.0
3,668581000000000000,Modern Apartment in Prime Location (ID8153X26),Sellwood-Moreland Improvement League,Entire rental unit,Entire home/apt,2,"[""Shampoo"", ""Bed linens"", ""Essentials"", ""Carbo...",$68.00,5.0
4,579018000000000000,Hidden Cove with Private Bath,Creston-Kenilworth,Private room in home,Private room,2,"[""Carbon monoxide alarm"", ""Refrigerator"", ""Bar...",$70.00,5.0
...,...,...,...,...,...,...,...,...,...
1741,719592000000000000,One bedroom in SE Portland w/plenty of local e...,Brentwood-Darlington,Private room in home,Private room,2,"[""Dedicated workspace"", ""Essentials"", ""Carbon ...",$49.00,NaN
1742,720564000000000000,Modern Apartment in Prime Location (ID9822X42),Buckman,Entire rental unit,Entire home/apt,2,"[""Shampoo"", ""Bed linens"", ""Essentials"", ""Carbo...",$79.00,NaN
1743,720935000000000000,One bedroom in a cozy SE home,Mt. Scott-Arleta,Private room in home,Private room,2,"[""Dedicated workspace"", ""Self check-in"", ""Carb...",$60.00,NaN
1744,721417000000000000,Let the barking dogs rest or run!,Russell,Private room in home,Private room,2,"[""Carbon monoxide alarm"", ""Refrigerator"", ""Bar...",$114.00,NaN
